## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals, param_to_list, list_to_param

# Mute warnings:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ax.plot.trace import optimization_trace_single_method

In [3]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [4]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [5]:
Tag_client = AxClient()

[INFO 04-10 17:14:23] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [6]:
# Define bounds for each parameter type
int_bounds = {
    "z_sh": [6, 12], #NOTE - z_sh is actually half z_s (which must be even)
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-0.8, 0.8],
    "x_r2": [-1., 1.],
    # Cl is fixed as : 0.10-0.40
}

paper_vals = [6, 81, .476, 1.0]

# Number of copies for each parameter
num_copies = 4

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (num_copies-1-i), upper - i],  # adjust bounds to deal with order
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (.1*(num_copies-1-i)), upper - .1*i],  # adjust bounds to deal with order
            "value_type": "float",
        })

# Add int / float parameters that make sure 

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Cross_term_gear_ratio_product": ObjectiveProperties(minimize=False)},
    parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-10 17:14:23] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[71, 84]), RangeParameter(name='z_r2_3', parameter_type=INT, range=[70, 83]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.5, 0.8]), RangeParameter(name='x_s_1', parameter_type=FLOAT, range=[-0.6000000000000001, 0.7000000000000001]), RangeParameter(name='x_s_2', parameter_type=FLOAT, range=[-0.7000000000000001, 0.6000000000000001]), RangeParameter(name='x_s_3', parameter_type=FLOAT, range=[-0.8, 0.5]), RangeParameter(name='x_r2', par

In [7]:
max_cross_terms = 4**(len(int_bounds) + len(float_bounds)+1) # plus Cl.
print(f'max_cross_terms: {max_cross_terms}')

max_cross_terms: 1024


In [8]:
# -1.0*z_p2 + 1.0*z_p2_1 <= -1.0
# -1.0*x_p2_1 + 1.0*x_p2 <= -0.01

In [9]:
parameters

[{'name': 'z_sh', 'type': 'range', 'bounds': [9, 12], 'value_type': 'int'},
 {'name': 'z_sh_1', 'type': 'range', 'bounds': [8, 11], 'value_type': 'int'},
 {'name': 'z_sh_2', 'type': 'range', 'bounds': [7, 10], 'value_type': 'int'},
 {'name': 'z_sh_3', 'type': 'range', 'bounds': [6, 9], 'value_type': 'int'},
 {'name': 'z_r2', 'type': 'range', 'bounds': [73, 86], 'value_type': 'int'},
 {'name': 'z_r2_1', 'type': 'range', 'bounds': [72, 85], 'value_type': 'int'},
 {'name': 'z_r2_2', 'type': 'range', 'bounds': [71, 84], 'value_type': 'int'},
 {'name': 'z_r2_3', 'type': 'range', 'bounds': [70, 83], 'value_type': 'int'},
 {'name': 'x_s',
  'type': 'range',
  'bounds': [-0.5, 0.8],
  'value_type': 'float'},
 {'name': 'x_s_1',
  'type': 'range',
  'bounds': [-0.6000000000000001, 0.7000000000000001],
  'value_type': 'float'},
 {'name': 'x_s_2',
  'type': 'range',
  'bounds': [-0.7000000000000001, 0.6000000000000001],
  'value_type': 'float'},
 {'name': 'x_s_3',
  'type': 'range',
  'bounds': [-

In [10]:
parameter_constraints

['z_sh - z_sh_1 >= 1',
 'z_sh_1 - z_sh_2 >= 1',
 'z_sh_2 - z_sh_3 >= 1',
 'z_r2 - z_r2_1 >= 1',
 'z_r2_1 - z_r2_2 >= 1',
 'z_r2_2 - z_r2_3 >= 1',
 'x_s - x_s_1 >= 0.1',
 'x_s_1 - x_s_2 >= 0.1',
 'x_s_2 - x_s_3 >= 0.1',
 'x_r2 - x_r2_1 >= 0.1',
 'x_r2_1 - x_r2_2 >= 0.1',
 'x_r2_2 - x_r2_3 >= 0.1']

### Run tests to solve for best possible score    

In [11]:
success = 0
fail_count = 0
while not success:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        success = 1
    except:
        fail_count += 1

print(f"Failed to get next trial {fail_count} times")

[WARNING 04-10 17:14:36] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 12.240282338935529, 'z_sh_1': 10.417266357167737, 'z_sh_2': 8.195437952050213, 'z_sh_3': 6.206021728951185, 'z_r2': 85.18254403903825, 'z_r2_1': 84.10801407113689, 'z_r2_2': 82.68728881433485, 'z_r2_3': 73.371163939481, 'x_s': 0.7004904272034764, 'x_s_1': 0.4276198435574772, 'x_s_2': -0.4646882610395551, 'x_s_3': -0.6758179585449398, 'x_r2': 0.855338825378567, 'x_r2_1': 0.03748058853670955, 'x_r2_2': -0.07896541664376844, 'x_r2_3': -0.48660729695111515}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), Ran

Failed to get next trial 36 times


In [12]:
parameterization

{'z_sh': 12,
 'z_sh_1': 10,
 'z_sh_2': 8,
 'z_sh_3': 6,
 'z_r2': 85,
 'z_r2_1': 84,
 'z_r2_2': 83,
 'z_r2_3': 73,
 'x_s': 0.7004904272034764,
 'x_s_1': 0.4276198435574772,
 'x_s_2': -0.4646882610395551,
 'x_s_3': -0.6758179585449398,
 'x_r2': 0.855338825378567,
 'x_r2_1': 0.03748058853670955,
 'x_r2_2': -0.07896541664376844,
 'x_r2_3': -0.48660729695111515}

In [13]:
print(len(parameterization))
for name, value in parameterization.items():
    print(f"{name}: {value}")

16
z_sh: 12
z_sh_1: 10
z_sh_2: 8
z_sh_3: 6
z_r2: 85
z_r2_1: 84
z_r2_2: 83
z_r2_3: 73
x_s: 0.7004904272034764
x_s_1: 0.4276198435574772
x_s_2: -0.4646882610395551
x_s_3: -0.6758179585449398
x_r2: 0.855338825378567
x_r2_1: 0.03748058853670955
x_r2_2: -0.07896541664376844
x_r2_3: -0.48660729695111515


In [14]:
Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True), add_gear = False))

[INFO 04-10 17:14:36] ax.service.ax_client: Completed trial 0 with data: {'Cross_term_gear_ratio_product': (np.float64(6259.106308), None)}.


In [15]:
list = param_to_list(parameterization, add_cl = True)
print(list)

[array([12, 10,  8,  6]), array([85, 84, 83, 73]), array([ 0.70049043,  0.42761984, -0.46468826, -0.67581796]), array([ 0.85533883,  0.03748059, -0.07896542, -0.4866073 ]), array([0.0001, 0.0002, 0.0003, 0.0004])]


NOTE - run the following through 'Exhausted' errors.

In [16]:
init_count = 0
init_trials = 32 # hand set based on output at initialization
fail_count = 0
while init_count < init_trials:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        # Local evaluation here can be replaced with deployment to external system.
        Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True), add_gear = False))
    except:
        # print('Not this time')
        fail_count += 1
    init_count = trial_index
print(f'Failed {fail_count} times while using Sobol')

[WARNING 04-10 17:14:39] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 11.825064538119696, 'z_sh_1': 10.766965882847739, 'z_sh_2': 9.009689885759336, 'z_sh_3': 6.470134758459926, 'z_r2': 80.7649616018415, 'z_r2_1': 76.25539451934694, 'z_r2_2': 72.58265984505591, 'z_r2_3': 69.89228620724593, 'x_s': 0.7880172297358514, 'x_s_1': 0.6277808016166093, 'x_s_2': 0.46550305709242845, 'x_s_3': -0.6107368575409056, 'x_r2': 0.7267206812277436, 'x_r2_1': 0.0882309740409255, 'x_r2_2': -0.2864908185787498, 'x_r2_3': -0.9796801106072962}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), Rang

Failed 1253 times while using Sobol


In [17]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Cross_term_gear_ratio_product,z_sh,z_sh_1,z_sh_2,z_sh_3,...,z_r2_2,z_r2_3,x_s,x_s_1,x_s_2,x_s_3,x_r2,x_r2_1,x_r2_2,x_r2_3
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,6259.106308,12,10,8,6,...,83,73,0.700490,0.427620,-0.464688,-0.675818,0.855339,0.037481,-0.078965,-0.486607
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,16117.843537,12,11,9,6,...,73,70,0.788017,0.627781,0.465503,-0.610737,0.726721,0.088231,-0.286491,-0.979680
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,9472.316119,11,10,7,6,...,79,74,0.376808,-0.069420,-0.172476,-0.333940,0.421677,-0.031791,-0.451806,-0.896414
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,15695.392196,12,10,8,7,...,73,72,0.469493,0.021546,-0.339447,-0.477784,0.163369,0.031503,-0.595263,-0.701654
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,15581.952334,12,11,8,7,...,74,73,0.629047,0.483614,-0.162645,-0.684574,0.676014,0.135554,-0.329515,-0.964646
5,5,5_0,COMPLETED,Sobol,GenerationStep_0,24062.260580,11,8,7,6,...,72,70,0.776482,0.419024,0.036212,-0.157088,0.450841,-0.030438,-0.610979,-0.932688
6,6,6_0,COMPLETED,Sobol,GenerationStep_0,14291.455583,12,10,8,6,...,74,73,-0.046928,-0.290557,-0.446709,-0.693606,0.998143,0.551621,-0.179981,-0.413204
7,7,7_0,COMPLETED,Sobol,GenerationStep_0,14953.581200,12,9,7,6,...,76,71,0.710900,0.400114,-0.253818,-0.542620,0.928930,0.168337,-0.271384,-0.472512
8,8,8_0,COMPLETED,Sobol,GenerationStep_0,9965.337450,12,9,8,6,...,77,70,0.311445,-0.175039,-0.311864,-0.553376,0.591556,0.169035,0.064595,-0.700013
9,9,9_0,COMPLETED,Sobol,GenerationStep_0,11397.315150,11,10,8,6,...,79,73,0.254370,0.116445,-0.531801,-0.765776,0.858634,0.086453,-0.208831,-0.372570


In [18]:
render(Tag_client.get_optimization_trace())

In [19]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True), add_gear = False))

[INFO 04-10 17:22:46] ax.service.ax_client: Generated new trial 33 with parameters {'z_sh': 11, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 75, 'z_r2_1': 74, 'z_r2_2': 73, 'z_r2_3': 72, 'x_s': 0.8, 'x_s_1': 0.625111, 'x_s_2': 0.119953, 'x_s_3': 0.019953, 'x_r2': 0.387131, 'x_r2_1': -0.37677, 'x_r2_2': -0.47677, 'x_r2_3': -0.73888} using model BoTorch.
[INFO 04-10 17:22:46] ax.service.ax_client: Completed trial 33 with data: {'Cross_term_gear_ratio_product': (np.float64(24828.250458), None)}.
[INFO 04-10 17:22:54] ax.service.ax_client: Generated new trial 34 with parameters {'z_sh': 12, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 74, 'z_r2_1': 73, 'z_r2_2': 72, 'z_r2_3': 70, 'x_s': 0.8, 'x_s_1': 0.642872, 'x_s_2': 0.159461, 'x_s_3': 0.059461, 'x_r2': 0.540316, 'x_r2_1': -0.236454, 'x_r2_2': -0.522809, 'x_r2_3': -1.0} using model BoTorch.
[INFO 04-10 17:22:54] ax.service.ax_client: Completed trial 34 with data: {'Cross_term_gear_ratio_product': (np.float64(22772.286763), None)}.
[I

In [20]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")

[INFO 04-10 17:39:07] ax.modelbridge.base: Leaving out out-of-design observations for arms: 125_0


z_sh: 9
z_sh_1: 8
z_sh_2: 7
z_sh_3: 6
z_r2: 74
z_r2_1: 73
z_r2_2: 72
z_r2_3: 71
x_s: 0.7999999999999999
x_s_1: 0.6999999999999924
x_s_2: -0.07585252513800794
x_s_3: -0.17585252513801006
x_r2: 1.0
x_r2_1: 0.8999999999999999
x_r2_2: 0.7999999999999988
x_r2_3: 0.6999999999999987


In [21]:
print(score_vals(param_to_list(best_parameters, add_cl = True), add_gear = False))

37247.97113756016


In [22]:
param_to_list(best_parameters, add_cl = True)

[array([9, 8, 7, 6]),
 array([74, 73, 72, 71]),
 array([ 0.8       ,  0.7       , -0.07585253, -0.17585253]),
 array([1. , 0.9, 0.8, 0.7]),
 array([0.0001, 0.0002, 0.0003, 0.0004])]

In [23]:
render(Tag_client.get_optimization_trace())

In [24]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-10 17:39:13] ax.modelbridge.base: Leaving out out-of-design observations for arms: 125_0
[INFO 04-10 17:39:27] ax.service.ax_client: Generated new trial 133 with parameters {'z_sh': 12, 'z_sh_1': 11, 'z_sh_2': 10, 'z_sh_3': 9, 'z_r2': 80, 'z_r2_1': 72, 'z_r2_2': 71, 'z_r2_3': 70, 'x_s': 0.8, 'x_s_1': 0.7, 'x_s_2': 0.6, 'x_s_3': -0.8, 'x_r2': 1.0, 'x_r2_1': 0.9, 'x_r2_2': 0.8, 'x_r2_3': -1.0} using model BoTorch.
[INFO 04-10 17:39:27] ax.service.ax_client: Completed trial 133 with data: {'Cross_term_gear_ratio_product': (np.float64(1086.574994), None)}.
[INFO 04-10 17:39:27] ax.modelbridge.base: Leaving out out-of-design observations for arms: 125_0
[INFO 04-10 17:39:39] ax.service.ax_client: Generated new trial 134 with parameters {'z_sh': 12, 'z_sh_1': 9, 'z_sh_2': 8, 'z_sh_3': 6, 'z_r2': 80, 'z_r2_1': 75, 'z_r2_2': 74, 'z_r2_3': 71, 'x_s': -0.5, 'x_s_1': -0.6, 'x_s_2': -0.7, 'x_s_3': -0.8, 'x_r2': -0.7, 'x_r2_1': -0.8, 'x_r2_2': -0.9, 'x_r2_3': -1.0} using model BoTorch.
[IN

In [25]:
render(Tag_client.get_optimization_trace())

In [26]:
for i in range(800):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-10 18:01:14] ax.modelbridge.base: Leaving out out-of-design observations for arms: 125_0
[INFO 04-10 18:01:26] ax.service.ax_client: Generated new trial 233 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 74, 'z_r2_1': 73, 'z_r2_2': 72, 'z_r2_3': 70, 'x_s': 0.8, 'x_s_1': 0.7, 'x_s_2': -0.321194, 'x_s_3': -0.421194, 'x_r2': 1.0, 'x_r2_1': 0.9, 'x_r2_2': 0.8, 'x_r2_3': 0.7} using model BoTorch.
[INFO 04-10 18:01:26] ax.service.ax_client: Completed trial 233 with data: {'Cross_term_gear_ratio_product': (np.float64(2817.374045), None)}.
[INFO 04-10 18:01:27] ax.modelbridge.base: Leaving out out-of-design observations for arms: 125_0
[INFO 04-10 18:01:39] ax.service.ax_client: Generated new trial 234 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 74, 'z_r2_1': 73, 'z_r2_2': 72, 'z_r2_3': 71, 'x_s': 0.8, 'x_s_1': 0.7, 'x_s_2': 0.08649, 'x_s_3': -0.01351, 'x_r2': 0.437148, 'x_r2_1': 0.337148, 'x_r2_2': 0.237148, 'x_r2_3': 0.13714

In [30]:
render(Tag_client.get_optimization_trace())

In [27]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")

[INFO 04-11 11:01:21] ax.modelbridge.base: Leaving out out-of-design observations for arms: 827_0, 981_0, 719_0, 125_0, 903_0, 841_0


z_sh: 9
z_sh_1: 8
z_sh_2: 7
z_sh_3: 6
z_r2: 74
z_r2_1: 73
z_r2_2: 72
z_r2_3: 71
x_s: 0.7999999999999999
x_s_1: 0.6999999999999924
x_s_2: -0.07585252513800794
x_s_3: -0.17585252513801006
x_r2: 1.0
x_r2_1: 0.8999999999999999
x_r2_2: 0.7999999999999988
x_r2_3: 0.6999999999999987


In [28]:
print(score_vals(param_to_list(best_parameters, add_cl = True), add_gear = False))

37247.97113756016


In [29]:
param_to_list(best_parameters, add_cl = True)

[array([9, 8, 7, 6]),
 array([74, 73, 72, 71]),
 array([ 0.8       ,  0.7       , -0.07585253, -0.17585253]),
 array([1. , 0.9, 0.8, 0.7]),
 array([0.0001, 0.0002, 0.0003, 0.0004])]